## Introduction
The purpose of this notebook is to show how to perform inference on quadrat test images with a tiling-based approach. As you may have have already noticed, predicting the species directly on the entire high-resolution image is a challenging task that often yields suboptimal results. The more intuitive strategy is then split the image into smaller patches, predict the species on each patch and finally aggregate the results.

Here we use a simple method where predictions are thresholded based on species scores and their relative positions among the softmax output. The predictions for each tile are generated using the DINOv2-based model provided in the competition.
This notebook can be used as a starting point for further development. Feel free to leave comments on errors or for any improvement.

### Import libraries

In [ ]:
import numpy as np
import pandas as pd
import timm
import torch
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import time
import os
import torchvision.transforms as T
from torch.amp import autocast
from matplotlib import pyplot as plt
from kornia import tensor_to_image
from kornia.contrib import extract_tensor_patches, compute_padding
import csv
import random

In [ ]:
class AverageMeter:
    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class PatchDataset(Dataset):
    def __init__(self, patches, transform=None):
        self.patches = patches.squeeze(0)
        self.transform = transform

    def __len__(self):
        return self.patches.size(0)

    def __getitem__(self, idx):
        patch = self.patches[idx]

        if self.transform:
            patch = self.transform(patch)
        return patch


class TestDataset(Dataset):
    def __init__(
        self, image_folder, patch_size=518, stride=259, transform=None, use_pad=False
    ):
        self.image_folder = image_folder
        self.image_paths = [
            os.path.join(image_folder, f) for f in os.listdir(image_folder)
        ]
        self.transform = transform
        self.use_pad = use_pad
        self.patch_size = patch_size
        self.stride = stride

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path)

        if self.transform:
            image = self.transform(image).unsqueeze(0)

        h, w = image.shape[-2:]

        if self.use_pad:
            pad = compute_padding(
                original_size=(h, w), window_size=self.patch_size, stride=self.stride
            )
            patches = extract_tensor_patches(
                image, self.patch_size, self.stride, padding=pad
            )
        else:
            patches = extract_tensor_patches(image, self.patch_size, self.stride)

        return patches, image_path

In [ ]:
PROJECT_PATH = "/mnt/storage1/shared_data/plant_clef_2025/"
DATA_FOLDER_PATH = f"{PROJECT_PATH}data/"
TEST_DATA_PATH = f"{DATA_FOLDER_PATH}plant_clef_2025_test/"
METADATA_PATH = f"{DATA_FOLDER_PATH}metadata/"

df_species_ids = pd.read_csv(f"{METADATA_PATH}species_ids.csv")

df_metadata = pd.read_csv(
    f"{METADATA_PATH}PlantCLEF2024_single_plant_training_metadata.csv",
    sep=";",
    dtype={"partner": str},
)
class_map = df_species_ids[
    "species_id"
].to_dict()  # dictionary to map the species model Id with the species Id

df_metadata.head()

In [ ]:
MODEL_FOLDER_PATH = f"{PROJECT_PATH}models/"


device = torch.device("cuda")
model = timm.create_model(
    "vit_base_patch14_reg4_dinov2.lvd142m",
    pretrained=False,
    num_classes=len(df_species_ids),
    checkpoint_path=f"{MODEL_FOLDER_PATH}model_onlyclassifier_then_all/model_best.pth.tar",
)
model = model.to(device)
model = model.eval()

In [ ]:
data_config = timm.data.resolve_model_data_config(model)
model_input_size, model_mean, model_std = (
    data_config["input_size"][1],
    data_config["mean"],
    data_config["std"],
)

Set hyperparameters:
* batch_size: size of batch of testing images
* top_k: keep best top_k results for each patch
* min_score: keep only classes with a score higher than min_score
* patch_size: size of patches. We will be using the DinoV2 image input size that is 518.
* stride: overlapping stride. We will be using half of the patch size.

Exemple of tiling on single image. Let's load a random image

In [ ]:
def get_random_images_from_directory(directory: str, num_images: int = 10) -> list[str]:
    # List all files in the directory
    all_files = os.listdir(directory)
    # Randomly select num_images from the list
    selected_images = random.sample(all_files, min(num_images, len(all_files)))
    # Return full paths
    return [os.path.join(directory, img) for img in selected_images]


images_paths = get_random_images_from_directory(TEST_DATA_PATH, num_images=10)
images = [Image.open(path) for path in images_paths]

# plot the images
plt.figure(figsize=(20, 20))
for i, image in enumerate(images):
    plt.subplot(5, 5, i + 1)
    plt.imshow(image)
    plt.axis("off")
plt.show()

In [ ]:
import torchvision.transforms as ttransforms


def get_random_data_augmentation() -> ttransforms.transforms:
    return ttransforms.Compose(
        [
            ttransforms.RandomResizedCrop(size=(518, 518)),
            ttransforms.RandomHorizontalFlip(),
            ttransforms.RandomVerticalFlip(),
            ttransforms.RandomPerspective(distortion_scale=0.2),
            ttransforms.RandomRotation(20),
            ttransforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
        ]
    )

In [ ]:
# plot the images
aug = get_random_data_augmentation()

plt.figure(figsize=(20, 20))
for i, image in enumerate(images):
    aug_image = aug(image)
    plt.subplot(5, 5, i + 1)
    plt.imshow(aug_image)
    plt.axis("off")
plt.show()

Let's now visualize the 64 patches extracted with a size of 518 (the same of model input)

In [ ]:
data_config = timm.data.resolve_model_data_config(model)
model_input_size, model_mean, model_std = (
    data_config["input_size"][1],
    data_config["mean"],
    data_config["std"],
)

Set hyperparameters:
* batch_size: size of batch of testing images
* top_k: keep best top_k results for each patch
* min_score: keep only classes with a score higher than min_score
* patch_size: size of patches. We will be using the DinoV2 image input size that is 518.
* stride: overlapping stride. We will be using half of the patch size.

In [ ]:
batch_size = 64
min_score = 0.1
top_k_tile = 2
patch_size = model_input_size
stride = int(model_input_size) * 2
use_pad = True

In [ ]:
image_to_tensor = T.ToTensor()

image_tensor = image_to_tensor(image).unsqueeze(0)
h, w = image_tensor.shape[-2:]

pad = compute_padding(original_size=(h, w), window_size=patch_size, stride=stride)

patches = extract_tensor_patches(image_tensor, patch_size, stride, padding=pad)
print(f"Shape of image tiles = {patches.shape}")

In [ ]:
import math


fig, axs = plt.subplots(
    int(math.sqrt(patches.shape[1])), int(math.sqrt(patches.shape[1]))
)
axs = axs.ravel()

for i in range(len(patches[0])):
    axs[i].axis("off")
    axs[i].imshow(tensor_to_image(patches[0][i]))

plt.show()

In [ ]:
# Example of a single patch
plt.imshow(tensor_to_image(patches[0][1]))
plt.axis("off")
plt.show()